This Notebook evaluates test cases for the 1D Shock-Acoustic-Wave interaction problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESTSF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESTSF;
using XDGShock;
Init();


In [2]:
var ass= new MatrixAssembler(1);
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 12; Grid Count = 12; Path = \\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D }

In [3]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D

function to plot line outs of the space time solution

In [4]:
(double[] xVals, double[]yVals) GetPlot(double xMin,double xMax, double t, int nRef,string field,ISessionInfo si){
//double shockPosition=0.5;
var p1= new double[] {xMin,t};
var p2=new double[] {xMax,t};

//timesteps to compare
var tiEnd=si.Timesteps.Last();

// # use p_per from BoSSS 
var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

In [5]:
var db=OpenOrCreateDatabase(dbPath);
var ses=db.Sessions;
ses

#0: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:38 PM	50cba6ca...
#1: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:01 PM	3fb780f8...
#2: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	4/10/2024 2:46:46 PM	9bcce582...
#3: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:46:07 PM	012246ad...
#4: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:45:27 PM	90107f8d...
#5: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	4/10/2024 2:47:24 PM	8c0c4d79...
#6: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5

In [7]:
var si=db.Sessions.Pick(0);
var ti=si.Timesteps.Last();
//si.Timesteps
si

XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:38 PM	50cba6ca...

# Plotting

define general params for plotting (must substract some eps because, otherwise some points are not located in the space-time grid)

In [8]:
var eps=0.000001;
double xMin=0 +eps;double xMax=3.0-eps; int xRef=100; 
double tMin =0 + eps;int tRef=100;
int nRows=2;

obtain tMax from session name

In [9]:
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;

## Pressure Pertubations

In [10]:
using System.Text.RegularExpressions;
foreach(var si in ses){
    string pattern = @"tMax(\d+)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }
    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    string field="p_per";
    string csvFileName = si.Name + "_p_per.csv";
    Plot2Ddata[,] multiplotsPressure = new Plot2Ddata[(tRef+1)/nRows,nRows];
    using (StreamWriter writer = new StreamWriter(csvFileName))
            {
            for(int Ti=0;Ti< tRef;Ti++){
                var plot= new Plot2Ddata();
                plot.Xlabel="x";
                plot.Ylabel="p'";
                double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
                //Console.WriteLine(t);
                (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
                plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
                multiplotsPressure[Ti/nRows,Ti %nRows]=plot;
                if (Ti == 0)
                {
                    // Write header row
                    writer.WriteLine($"x/t,{string.Join(",", xVals)}");
                }
                // Write data row
                writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                        
            }
    };
}



doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_a

In [11]:
foreach(var si in ses){
    IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)si.Timesteps.Last()).GetInternal();
    lts.EnResHistory.SaveToTextFile(si.Name + "_EnResNorms.txt");
    lts.ResHistory.SaveToTextFile(si.Name + "_ResNorms.txt");
}
//Array.ForEach(multiplotsPressure.Cast<Plot2Ddata>().ToArray(), 
//    plot => { plot.YrangeMin = -1.5 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
// multiplotsPressure[1,1].YrangeMax = 5 * 1e-7; multiplotsPressure[1,1].YrangeMin = -6 * 1e-7;
// multiplotsPressure[2,0].YrangeMax = 5 * 1e-7; multiplotsPressure[2,0].YrangeMin = -6 * 1e-7;
// multiplotsPressure[2,1].YrangeMax = 5 * 1e-7; multiplotsPressure[2,1].YrangeMin = -6 * 1e-7;
//multiplotsPressure.ToGnuplot().PlotSVG(xRes:900,yRes:600)

In [18]:
using BoSSS.Solution.CompressibleFlowCommon;
double densityL=1, pressureL=1,MachL=1.5,gamma=1.4;
double cL = Math.Sqrt(gamma * pressureL / densityL);
double velocityL = MachL * cL;
(double densityR, double velocityR, double pressureR, double cR, double MachR)
    = CompressibleHelperFunc.ComputeNormalShockWaveRelations(densityL, velocityL, pressureL, MachL, gamma);
(double[] xVals, double[]yVals) GetPlotAlongRay(double xMax, int nRef,string field,ISessionInfo si, double xa, double xs){
//double shockPosition=0.5;
var inc_speed= xa<xs? (velocityL + cL): (velocityR - cR);
var tstar= (xs-xa)/inc_speed;
Func<double,double> X_s = t => (velocityR + cR)*t+ (xs-tstar*(velocityR + cR));
double tEnd= (xMax - (xs-tstar*(velocityR + cR)))/(velocityR + cR);
var eps=5e-2;
var p1= new double[] {X_s(tstar+eps),tstar+eps};
var p2=new double[] {xMax,tEnd};

//timesteps to compare
var tiEnd=si.Timesteps.Last();

// # use p_per from BoSSS 
var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(tstar,tEnd,nRef);
return(xVals,yVals);
}

Plot2Ddata[,] multiplotsPressure = new Plot2Ddata[1,1];
foreach(var si in ses){
    //catch tmax value
    string pattern = @"tMax(\d+(\.\d+)?)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }

    // catch shock position
    pattern = @"sP(\d+(\.\d+)?)";
    Match match2 = Regex.Match(si.Name, pattern);
    double xs = 1.5;
    try{
        xs = double.Parse(match2.Groups[1].Value);
    }catch{
    }

    // catch wave positions
    pattern = @"wP(\d+(\.\d+)?)";
    match = Regex.Match(si.Name, pattern);
    double wP = 0.0;
    try{
        wP = double.Parse(match.Groups[1].Value);
    }catch{
    }

    // catch wave length and convert to position of amplitude
    pattern = @"wL(\d+(\.\d+)?)";
    match = Regex.Match(si.Name, pattern);
    double xa = 1.5;
    try{
        if(si.Name.Contains("1sinus")){
            xa = wP+double.Parse(match.Groups[1].Value)/4;
        }else{
            xa = wP+double.Parse(match.Groups[1].Value)/2;
        }
        
    }catch{
    }




    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    Console.WriteLine($"Catched: wP{wP}, xa{xa},xs{xs}");
    string field="p_per";
    string csvFileName = si.Name + "_p_per_Ray.csv";
    using (StreamWriter writer = new StreamWriter(csvFileName))
        {
            var plot= new Plot2Ddata();
            plot.Xlabel="x";
            plot.Ylabel="p'";
            //Console.WriteLine(t);
            (double[] xVals,double[] yVals)=GetPlotAlongRay(xMax,xRef,field, si,xa,xs);
            Console.WriteLine($"AverageAlongRay: {yVals.Average()}");
            plot.AddDataGroup("Ray",xVals,yVals,fmt);
            multiplotsPressure[0,0]=plot;
            // Write header row
            writer.WriteLine($"x/t,{string.Join(",", xVals)}");
            
            // Write data row
            writer.WriteLine($"Ray,{string.Join(",", yVals)}");
                    
        }
};

multiplotsPressure.ToGnuplot().PlotSVG(xRes:900,yRes:600)
//    plot => { plot.YrangeMin = -1.5 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });A


doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
Catched: wP0.3, xa0.5,xs1.5
AverageAlongRay: -3.2634017135110644E-10
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
Catched: wP0.3, xa0.7,xs1.5
AverageAlongRay: -7.1809644682885924E-12
doing:AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:1.499999
Catched: wP0, xa0.4,xs1.5
AverageAlongRay: 2.1574504113916715E-05
doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
Catched: wP0.9, xa1.1,xs0.5
AverageAlongRay: -9.660974521687129E-07
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
Catched: wP0.9, xa1.3,xs0.5
AverageAlongRay: 2.867831112623626E-07
doing:AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1x10 -6 
 
 
 
 
 0 
 
 
 
 
 1x10 -6 
 
 
 
 
 2x10 -6 
 
 
 
 
 3x10 -6 
 
 
 
 
 4x10 -6 
 
 
 
 
 5x10 -6 
 
 
 
 
 6x10 -6 
 
 
 
 
 7x10 -6 
 
 
 
 
 8x10 -6 
 
 
 
 
 9x10 -6 
 
 
 
 
 1.8 
 
 
 
 
 2 
 
 
 
 
 2.2 
 
 
 
 
 2.4 
 
 
 
 
 2.6 
 
 
 
 
 2.8 
 
 
 
 
 3 
 
 
 
 
 3.2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 p' 
 
 
 
 
 x 
 
 
 
 
 Ray 
 
 
 Ray 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M735.0,63.1 L788.4,63.1 M187.0,60.6 L192.8,90.6 L198.6,129.0 L204.4,175.9 L210.2,231.5 L216.0,298.9
 L221.8,376.2 L227.6,445.0 L233.4,485.5 L239.2,496.6 L245.0,495.1 L250.8,493.9 L256.6,494.2 L262.4,494.5
 L268.2,494.4 L274.0,494.4 L279.8,494.4 L285.6,494.4 L291.4,494.4 L297.2,494.4 L303.0,494.4 L308.8,494.4
 L314.6,494.4 L320.4,494.4 L326.2,494.4 L332.0,494.4 L337.8,494.4 L343.6,494.4 L349.4,494.4 L355.2,494.4
 L361.0,494.4 L366.8,494.4 L372.6,494.4 L378.4,494.4 L384.2,494.4 L390.0,494.4 L395.8,494.4 L401.6,494.4
 L407.4,494.4 L413.2,494.4 L419.0,494.4 L424.8,494.4 L430.6,494.4 L436.4,494.4 L442.2,494.4 L448.0,494.4
 L453.8,494.4 L459.6,494.4 L465.4,494.4 L471.2,494.4 L477.0,494.4 L482.8,494.4 L488.6,494.4 L494.4,494.4
 L500.2,494.4 L506.0,494.4 L511.8,494.4 L517.6,494.4 L523.4,494.4 L529.2,494.4 L535.0,494.4 L540.8,494.4
 L546.6,494.4 L552.4,494.4 L558.2,494.4 L564.0,494.4 L569.8,494.4 L575.6,494.4 L581.4,494.4 L587.2,494.4
 L593.0,494.4 L598.8,494.4 L604.6,494.4 L610.4,494.4 L616.2,494.4 L622.0,494.4 L627.8,494.4 L633.6,494.4
 L639.4,494.4 L645.2,494.4 L651.0,494.4 L656.8,494.4 L662.6,494.4 L668.4,494.4 L674.2,494.4 L680.0,494.4
 L685.8,494.4 L691.6,494.4 L697.4,494.4 L703.2,494.4 L709.0,494.4 L714.8,494.4 L720.6,494.4 L726.4,494.4
 L732.2,494.4 L738.0,494.4 L743.8,494.4 L749.6,494.4 L755.4,494.4 L761.2,494.4 '/>

## Density Pertubations

In [12]:
foreach(var si in ses){
        string pattern = @"tMax(\d+)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }
    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    string field="rho_per";
    // Format
    var fmt= new PlotFormat();
    fmt.WithPointSize(0.0);
    fmt.LineColor=LineColors.Red;

    Plot2Ddata[,] multiplotsDensity = new Plot2Ddata[(tRef+1)/nRows,nRows];
    string csvFileName = si.Name + "_rho_per.csv";
    using (StreamWriter writer = new StreamWriter(csvFileName)){
        for(int Ti=0;Ti< tRef;Ti++){
            var plot= new Plot2Ddata();
            plot.Xlabel="x";
            plot.Ylabel="rho'";
            double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
            //Console.WriteLine(t);
            (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
            plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
            multiplotsDensity[Ti/nRows,Ti %nRows]=plot;
            if (Ti == 0)
            {
                // Write header row
                writer.WriteLine($"x/t,{string.Join(",", xVals)}");
            }
            // Write data row
            writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                    
        }
        
    };                                              
}

doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_a

In [13]:
// Array.ForEach(multiplotsDensity.Cast<Plot2Ddata>().ToArray(), 
//     plot => { plot.YrangeMin = -1 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
// // multiplotsDensity[1,1].YrangeMax = 5 * 1e-7; multiplotsDensity[1,1].YrangeMin = -6 * 1e-7;
// //multiplotsDensity[2,0].YrangeMax = 5 * 1e-7; multiplotsDensity[2,0].YrangeMin = -6 * 1e-7;
// //multiplotsDensity[2,1].YrangeMax = 5 * 1e-7; multiplotsDensity[2,1].YrangeMin = -6 * 1e-7;
// multiplotsDensity.ToGnuplot().PlotSVG(xRes:1000,yRes:600)

In [15]:
(double[] xVals, double[]yVals) GetPlotLevelSet(double tMin,double tMax, int nRef,ISessionInfo si){
//double shockPosition=0.5;
var p1= new double[] {0.0,tMin};
var p2=new double[] {0.0,tMax};

//timesteps to compare
var tiEnd=si.Timesteps.Last();
// # use p_per from BoSSS 
var pEnd =tiEnd.GetField("LevelSet");
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

In [16]:
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;
string csvFileName = "XESTSFLevelSetPertubations.csv";
var plotLevelSet= new Plot2Ddata();
using (StreamWriter writer = new StreamWriter(csvFileName)){
    
    //Console.WriteLine(t);
    (double[] xVals,double[] yVals)=GetPlotLevelSet(tMin,tMax,1000, si);
    
    plotLevelSet.AddDataGroup("LevelSet",xVals,yVals,fmt);
        // Write header row
    writer.WriteLine($"t,{string.Join(",", xVals)}");
    
    // Write data row
    writer.WriteLine($"phi(t),{string.Join(",", yVals)}");
            
}

plotLevelSet.Ylabel="\varphi(0,t)";
plotLevelSet.ToGnuplot().PlotSVG(xRes:800,yRes:800)  


Error: (10,58): error CS0103: The name 'tMax' does not exist in the current context